**梯度裁剪（Gradient Clipping）**

动机就是防止“梯度爆炸”。当反传得到的梯度范数很大时，更新步长就会异常大，训练发散。

- 按范数裁剪（clip-by-norm）：设阈值 $\tau$，若 $\lVert g \rVert > \tau$，就把 $g \leftarrow g \cdot \frac{\tau}{\lVert g \rVert}$，方向不变、长度收缩；即 `torch.nn.utils.clip_grad_norm_(params, max_norm)`；
- 按数值裁剪（clip-by-value）：逐元素把梯度夹在 $[-\alpha, \alpha]$ 内；即 `torch.nn.utils.clip_grad_value_(params, clip_value)`；

Clip-Norm 会把整体范数压到阈值附近；Clip-Value 会限制逐元素幅度，但整体范数不一定接近阈值。

In [ ]:
import torch, torch.nn as nn
from torch.nn.utils import clip_grad_norm_, clip_grad_value_

torch.manual_seed(0)

# create a deep linear network with large gradients
layers = [nn.Linear(128, 128, bias=False) for _ in range(20)]
model = nn.Sequential(*layers)
for m in model:  # create large gradients by large weights
    # nn.init.normal_(m.weight, mean=0.0, std=0.5)
    nn.init.kaiming_normal_(m.weight)

x = torch.randn(32, 128)
y = torch.randn(32, 128)
opt = torch.optim.SGD(model.parameters(), lr=1e-2)

opt.zero_grad(set_to_none=True)
loss = (model(x) - y).pow(2).mean()
loss.backward()

# statistics of original gradients
all_params = [p for p in model.parameters() if p.grad is not None]
grad_norm = torch.norm(torch.stack([p.grad.detach().norm() for p in all_params]))
grad_absmax = max(p.grad.detach().abs().max().item() for p in all_params)
print(f"[Before] grad_norm={grad_norm:.3f}, absmax={grad_absmax:.3f}")

# clip-by-norm
clip_grad_norm_(model.parameters(), max_norm=1.0)
grad_normA = torch.norm(torch.stack([p.grad.detach().norm() for p in all_params]))
grad_absmaxA = max(p.grad.detach().abs().max().item() for p in all_params)
print(f"[Clip-Norm] grad_norm={grad_normA:.3f}, absmax={grad_absmaxA:.3f}")

# recompute gradients for clip-by-value
opt.zero_grad(set_to_none=True)
(model(x) - y).pow(2).mean().backward()

# clip-by-value
clip_grad_value_(model.parameters(), clip_value=0.01)
grad_normB = torch.norm(torch.stack([p.grad.detach().norm() for p in all_params]))
grad_absmaxB = max(p.grad.detach().abs().max().item() for p in all_params)
print(f"[Clip-Value] grad_norm={grad_normB:.3f}, absmax={grad_absmaxB:.3f}")

# take a step update, observe no divergence
opt.step()

[Before] grad_norm=4869029.500, absmax=95971.703
[Clip-Norm] grad_norm=1.000, absmax=0.020
[Clip-Value] grad_norm=5.724, absmax=0.010


**AMP 里的损失缩放（Loss Scaling）**


为避免 FP16 梯度下溢，先把 loss 乘以一个较大系数 $\alpha$ 再反传，得到的梯度也会被放大。

在 optimizer.step() 前再用 scaler.unscale_ 还原，从而既避免下溢，又不改变真实更新幅度。GradScaler 会动态调整 $\alpha$。

In [7]:
import torch, torch.nn as nn
from torch.nn.utils import clip_grad_norm_

model = nn.Sequential(nn.Linear(4096, 4096), nn.ReLU(), nn.Linear(4096, 4096)).to('mps')
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
scaler = torch.amp.GradScaler()

for step in range(3):
    x = torch.randn(32, 4096, device="mps")
    y = torch.randn(32, 4096, device="mps")
    opt.zero_grad(set_to_none=True)

    with torch.amp.autocast(device_type='mps', dtype=torch.float16):
        loss = (model(x) - y).pow(2).mean()

    # 1) the gradients are "amplified"
    scaler.scale(loss).backward()

    # 2) unscale the gradients to the real scale
    scaler.unscale_(opt)

    # 3) clip-by-norm (e.g., threshold 1.0)
    total_norm = clip_grad_norm_(model.parameters(), max_norm=1.0)
    if torch.isfinite(total_norm):
        print(f"step={step}, total_grad_norm(after clip)={total_norm.item():.3f}")

    # 4) update the model
    scaler.step(opt)
    scaler.update()

step=0, total_grad_norm(after clip)=0.225
step=1, total_grad_norm(after clip)=0.377
step=2, total_grad_norm(after clip)=0.297
